In [34]:
import os
import cv2
import numpy as np
import matplotlib.pyplot as plt
import cv2

In [2]:
#file path
directory="your dataset file path"
categories=['button','checkbox','dropdown','image','table'] # I chose these five elemenet from the whole data set 

In [3]:
#reading images
img_size=120
data= []
for category in categories:
    folder=os.path.join(directory,category)
    label= categories.index(category)
    for img in os.listdir(folder):
        img_path=os.path.join(folder, img)
        img_arr= cv2.imread(img_path)
        img_arr = cv2.cvtColor(img_arr, cv2.COLOR_BGR2GRAY)
        img_arr= cv2.resize(img_arr,(img_size,img_size))
        data.append([img_arr, label])

In [4]:
from sklearn.model_selection import train_test_split
# Split the data into training and testing sets
train_data, test_data = train_test_split(data, test_size=0.2, random_state=42)

# Extract the features and labels from the training and testing sets
train_images, train_labels = zip(*train_data)
test_images, test_labels = zip(*test_data)

In [5]:
x=np.array(train_images)
y=np.array(train_labels)
x=x/255 #normalizing images

## Building model

In [6]:
import tensorflow as tf
from keras.models import Sequential 
from keras.layers import Conv2D, MaxPooling2D, Flatten , Dense, Dropout, AveragePooling2D

In [7]:
# Define the model
model = Sequential()

model.add(Conv2D(64, (3, 3), activation='relu', input_shape=(img_size, img_size, 1)))

model.add(MaxPooling2D((2, 2)))

model.add(Conv2D(64, (3, 3), activation='relu'))

model.add(MaxPooling2D((2, 2)))

model.add(Flatten())

model.add(Dense(128, activation='relu'))

model.add(Dropout(0.4))

model.add(Dense(6, activation='softmax'))

# Compile the model
model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy']) 

In [8]:
# Build the model
model.build(input_shape=(None, 120, 120, 1))

model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d (Conv2D)             (None, 118, 118, 64)      640       
                                                                 
 max_pooling2d (MaxPooling2D  (None, 59, 59, 64)       0         
 )                                                               
                                                                 
 conv2d_1 (Conv2D)           (None, 57, 57, 64)        36928     
                                                                 
 max_pooling2d_1 (MaxPooling  (None, 28, 28, 64)       0         
 2D)                                                             
                                                                 
 flatten (Flatten)           (None, 50176)             0         
                                                                 
 dense (Dense)               (None, 128)               6

In [9]:
model.fit(x,y,epochs=8,validation_split=0.1)

Epoch 1/8
114/114 [==============================] - 82s 696ms/step - loss: 1.0251 - accuracy: 0.6143 - val_loss: 0.6420 - val_accuracy: 0.7778
Epoch 2/8
114/114 [==============================] - 78s 686ms/step - loss: 0.5418 - accuracy: 0.8077 - val_loss: 0.5024 - val_accuracy: 0.8296
Epoch 3/8
114/114 [==============================] - 78s 686ms/step - loss: 0.3676 - accuracy: 0.8743 - val_loss: 0.4632 - val_accuracy: 0.8420
Epoch 4/8
114/114 [==============================] - 78s 682ms/step - loss: 0.2705 - accuracy: 0.9045 - val_loss: 0.3978 - val_accuracy: 0.8741
Epoch 5/8
114/114 [==============================] - 77s 680ms/step - loss: 0.1782 - accuracy: 0.9405 - val_loss: 0.4485 - val_accuracy: 0.8370
Epoch 6/8
114/114 [==============================] - 77s 680ms/step - loss: 0.1390 - accuracy: 0.9545 - val_loss: 0.4441 - val_accuracy: 0.8519
Epoch 7/8
114/114 [==============================] - 77s 678ms/step - loss: 0.1090 - accuracy: 0.9668 - val_loss: 0.4615 - val_accuracy:

In [10]:
#test folder path
test_directory="file_path"
test_categories=['button','checkbox','dropdown','image','table']

In [11]:
test_data= []
size=120
for category in test_categories:
    folder=os.path.join(test_directory,category)
    label= test_categories.index(category)
    for img in os.listdir(folder):
        test_img_path=os.path.join(folder, img)
        test_img_arr= cv2.imread(test_img_path)
        test_img_arr = cv2.cvtColor(test_img_arr, cv2.COLOR_BGR2GRAY)
        test_img_arr= cv2.resize(test_img_arr,(size,size))
        test_data.append([test_img_arr, label])

In [12]:
X_test=np.array(test_images)
Y_test=np.array(test_labels)
X_test=X_test/255

## Evaluate the model on the test data

In [13]:
loss, accuracy = model.evaluate(X_test, Y_test)

32/32 [==============================] - 6s 186ms/step - loss: 0.4435 - accuracy: 0.9052


In [14]:
print("Test Loss:", loss*100)
print("Test Accuracy:", accuracy*100)

Test Loss: 44.35294568538666
Test Accuracy: 90.52320122718811


## predicting image

In [15]:
#image path
pred_image_path = "prediction_image.png" 

# Read the image
pred_image = cv2.imread(pred_image_path)

# Convert the image to grayscale
pred_image = cv2.cvtColor(pred_image, cv2.COLOR_BGR2GRAY)

# Resize the image to the desired size
pred_image = cv2.resize(pred_image, (img_size, img_size))

# Reshape the image for prediction
pred_image= np.reshape(pred_image, (1, 120, 120, 1))

# Normalize the image pixel values (if necessary)
#pred_image = pred_image / 255.0  # Normalize pixel values between

In [17]:
# Assuming you have a trained model stored in the 'model' variable
predictions = model.predict(pred_image)

# Get the predicted label/index
predicted_label_index = np.argmax(predictions)

# Assuming you have a list of category labels
predicted_label = categories[predicted_label_index]

# Print the predicted label
print("Predicted label:", predicted_label)


1/1 [==============================] - 0s 297ms/step
Predicted label: table


## saving model

In [18]:
#model.save('savemodel.h5')

## Creating Classification Report

In [19]:
from sklearn.metrics import classification_report

In [20]:
# Make predictions on the test data
y_pred = model.predict(X_test)
y_pred_labels = np.argmax(y_pred, axis=1)

# Generate the classification report
report = classification_report(Y_test, y_pred_labels, target_names=categories)

# Print the classification report
print(report)

32/32 [==============================] - 6s 187ms/step
              precision    recall  f1-score   support

      button       0.94      0.84      0.89       247
    checkbox       0.86      0.93      0.89       210
    dropdown       0.89      0.89      0.89       174
       image       0.94      0.95      0.95       213
       table       0.89      0.93      0.91       169

    accuracy                           0.91      1013
   macro avg       0.90      0.91      0.91      1013
weighted avg       0.91      0.91      0.91      1013



## predect an image with more than one element

In [44]:

# Load the image
image_path = 'elements.jpg'
img = cv2.imread(image_path, 0)  # Load as grayscale

# Apply Gaussian blur to reduce noise and improve edge detection
blurred = cv2.GaussianBlur(img, (5, 5), 0)

# Apply Canny edge detection
edges = cv2.Canny(blurred, 50, 150)

# Find contours in the edged image
contours, _ = cv2.findContours(edges, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)

threshold=5000 #change threshold if you wanted

filtered_contours = [contour for contour in contours if cv2.contourArea(contour) > threshold]

# Count the total number of shapes
num_shapes = len(filtered_contours)

print(f'Total Number of Shapes: {num_shapes}')



Total Number of Shapes: 4


In [45]:
# Draw bounding boxes around each detected contour and crop them
for i, contour in enumerate(filtered_contours):
        x, y, w, h = cv2.boundingRect(contour)
        cv2.rectangle(img, (x, y), (x + w, y + h), (0, 255, 0), 2)
   
        # Crop the box and save as a separate image
        cropped_box = img[y:y+h, x:x+w]
        cv2.imwrite(f'result_{i}.jpg', cropped_box)

In [46]:
# Define the directory where the cropped images are saved
box_directory = 'direcrtory path where images will be saved'

# Get a list of all files in the directory
box_files = [f for f in os.listdir(box_directory) if f.startswith('result_')]

# Sort the files based on the box number (i)
box_files.sort(key=lambda x: int(x.split('_')[1].split('.')[0]))

# Initialize an empty list to store the cropped boxes
cropped_boxes = []
predicted_label=[]
percentage_list=[]
# Get the class labels
class_labels = ['button','checkbox','dropdown','image','table'] #hia hia el categories 
# Load and append each cropped box to the list
for box_file in box_files:
    box_path = os.path.join(box_directory, box_file)
    cropped_box = cv2.imread(box_path)
    # Convert the image to grayscale
    cropped_box = cv2.cvtColor(cropped_box, cv2.COLOR_BGR2GRAY)

    # Resize the image
    cropped_box = cv2.resize(cropped_box, (img_size, img_size))
    # Reshape the image for prediction
    cropped_box= np.reshape(cropped_box, (1, 120, 120, 1))

    cropped_boxes.append(cropped_box)
    # Assuming you have a trained model stored in the 'model' variable
    predictions = model.predict(cropped_box)

    # Get the predicted label/index
    predicted_label_index = np.argmax(predictions)
    
    predicted_label.append(categories[predicted_label_index])

# Print the predicted label
print("Predicted label:", predicted_label)

1/1 [==============================] - 0s 31ms/step
Predicted label: ['table', 'checkbox', 'table', 'dropdown']
Predicted label probability: [100.0, 100.0, 100.0, 99.9998688697815]


## delete the saved image 

In [43]:
#box_files = [f for f in os.listdir(box_directory) if f.startswith('box_')]
#for box_file in box_files:
 #   box_path = os.path.join(box_directory, box_file)
  #  os.remove(box_path)

## create html code according to the predicted elements

In [ ]:
def generate_html_elements(element_types):
    html_code = """
<!DOCTYPE html>
<html lang="en">
<head>
    <meta charset="UTF-8">
    <meta name="viewport" content="width=device-width, initial-scale=1, shrink-to-fit=no">
    <title>Dynamic Elements</title>
    <link href="https://cdn.jsdelivr.net/npm/bootstrap@5.3.2/dist/css/bootstrap.min.css" rel="stylesheet" integrity="sha384-T3c6CoIi6uLrA9TneNEoa7RxnatzjcDSCmG1MXxSR1GAsXEV/Dwwykc2MPK8M2HN" crossorigin="anonymous">
</head>
<body>
"""

    for element_type in element_types:
        if element_type == 'button':
            html_code += """<a class="btn btn-primary" href="#" role="button">Link</a>
                        <input class="btn btn-primary" type="button" value="Input">"""
        elif element_type == 'checkbox':
            html_code += '<input type="checkbox"> Checkbox<br>'
        elif element_type == 'table':
            html_code += """
<table class="table table-dark table-striped">
    <thead>
        <tr>
            <th>ID</th>
            <th>Name</th>
            <th>Age</th>
            <th>Email</th>
        </tr>
    </thead>
    <tbody>
        <tr>
            <td>1</td>
            <td>John Doe</td>
            <td>30</td>
            <td>john@example.com</td>
        </tr>
        <tr>
            <td>2</td>
            <td>Jane Doe</td>
            <td>28</td>
            <td>jane@example.com</td>
        </tr>
        <tr>
            <td>3</td>
            <td>Jim Smith</td>
            <td>35</td>
            <td>jim@example.com</td>
        </tr>
    </tbody>
</table>
"""
        elif element_type == 'image':
            html_code += '<img src="" class="rounded mx-auto d-block" alt="Image">'
        elif element_type == 'dropdown':
            html_code += """
<div class="dropdown">
  <a class="btn btn-secondary dropdown-toggle" href="#" role="button" data-bs-toggle="dropdown" aria-expanded="false">
    Dropdown link
  </a>

  <ul class="dropdown-menu">
    <li><a class="dropdown-item" href="#">Action</a></li>
    <li><a class="dropdown-item" href="#">Another action</a></li>
    <li><a class="dropdown-item" href="#">Something else here</a></li>
  </ul>
</div>
"""
        else:
            return 'Invalid input. Please choose from "button", "checkbox", "table", "image", or "dropdown".'
    # Add closing tags for body and html sections
    html_code += """
    <script src="https://cdn.jsdelivr.net/npm/bootstrap@5.3.2/dist/js/bootstrap.bundle.min.js" integrity="sha384-C6RzsynM9kWDrMNeT87bh95OGNyZPhcTNXj1NW7RuBCsyN/o0jlpcV8Qyq46cDfL" crossorigin="anonymous"></script>
</body>
</html>
"""

    return html_code

In [ ]:
# Get user input for the types of elements
element_types = predicted_label

# Generate HTML code
html_code = generate_html_elements(element_types)

# Save the HTML code to a file
if not html_code.startswith('Invalid'):
    with open('generated_elements.html', 'w') as file:
        file.write(html_code)

print(f"Generated HTML code:\n{html_code}")